In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv("../datasets/train_radiomics_hipocamp.csv")


X_test = pd.read_csv("../datasets/test_radiomics_hipocamp.csv")

In [4]:
df['Transition'] = df['Transition'].map({
    'CN-CN': 0,
    'CN-MCI': 1,
    'MCI-MCI': 2,
    'MCI-AD': 3,
    'AD-AD': 4
})
print(df['Transition'])

0      0
1      0
2      4
3      1
4      0
      ..
300    0
301    0
302    3
303    2
304    0
Name: Transition, Length: 305, dtype: int64


In [5]:
X = df.drop(['Transition'], axis=1)     
y = df['Transition'].to_frame()  

In [6]:
X_numerico = X.select_dtypes(include=[np.number])

X_test = X_test.select_dtypes(include=[np.number])

In [7]:
from sklearn.feature_selection import VarianceThreshold
import pandas as pd

# Supondo que X_numerico e X_test sejam DataFrames do Pandas

# Aplicar o filtro de variância no conjunto de treino (X_numerico)
variance_filter = VarianceThreshold(threshold=0.05)
X_numerico_filtered = variance_filter.fit_transform(X_numerico)

# Obter as colunas que foram mantidas após o filtro
filtered_columns = X_numerico.columns[variance_filter.get_support()]

# Criar um novo DataFrame com as colunas filtradas
X_numerico = pd.DataFrame(X_numerico_filtered, columns=filtered_columns)

# Aplicar o filtro de variância no conjunto de teste (X_test)
X_test_filtered = variance_filter.transform(X_test)

# Criar um novo DataFrame para X_test com as colunas filtradas
X_test = pd.DataFrame(X_test_filtered, columns=filtered_columns)

# Exibir as primeiras linhas do novo DataFrame
print(X_numerico.head())
print(X_test.head())

# Exibir a quantidade de features (colunas) nos dois conjuntos
print(f"X_numerico_filtered_df - Número de features: {X_numerico.shape[1]}")
print(f"X_test_filtered_df - Número de features: {X_test.shape[1]}")


   diagnostics_Image-original_Mean  diagnostics_Image-original_Maximum  \
0                         5.848123                               178.0   
1                         5.238834                               190.0   
2                         6.816667                               155.0   
3                         6.445162                               171.0   
4                         5.568269                               150.0   

   diagnostics_Mask-original_VoxelNum  original_shape_LeastAxisLength  \
0                              7106.0                       10.757474   
1                              7779.0                       10.432362   
2                              6707.0                       14.370911   
3                              8281.0                       10.639815   
4                              5986.0                        8.885188   

   original_shape_MajorAxisLength  original_shape_Maximum2DDiameterColumn  \
0                      100.964915      

In [8]:
# Supondo que X_numerico e X_test sejam inicialmente DataFrames
X_numerico_df = pd.DataFrame(X_numerico)
X_test_df = pd.DataFrame(X_test)

# Criar uma matriz de correlação
correlation_matrix = X_numerico_df.corr().abs()

# Selecionar features com alta correlação
upper_triangle = np.triu(correlation_matrix, k=1)
correlated_features = [i for i in range(correlation_matrix.shape[1]) if any(upper_triangle[:, i] > 0.95)]

# Remover features altamente correlacionadas (mantenha como DataFrame)
X_numerico_df = X_numerico_df.drop(X_numerico_df.columns[correlated_features], axis=1)
X_test_df = X_test_df.drop(X_test_df.columns[correlated_features], axis=1)

# Relatório das dimensões
print(f"Número inicial de features: {X_numerico.shape[1]}")
print(f"Após filtragem por variância: {X_numerico_df.shape[1]}")
print(f"Após filtragem por correlação: {X_numerico_df.shape[1]}")
print(f"Após filtragem por correlação: {X_test_df.shape[1]}")
print(type(X_numerico_df))


Número inicial de features: 955
Após filtragem por variância: 501
Após filtragem por correlação: 501
Após filtragem por correlação: 501
<class 'pandas.core.frame.DataFrame'>


In [8]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = MinMaxScaler()
X_numerico = pd.DataFrame(scaler.fit_transform(X_numerico), columns=X_numerico.columns, index=X_numerico.index) #normalização

X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns, index=X_test.index)

In [107]:
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV



# ================= XGBoost =================
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, colsample_bytree= 0.9847923138822793, 
                    gamma= 3.8756641168055728, 
                    learning_rate= 0.29184968246925674, 
                    max_depth= 6, 
                    n_estimators = 369, 
                    subsample= 0.8636359979282104 )

# Espaço de hiperparâmetros para RandomSearchCV
xgb_param_dist = {
    'n_estimators': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 20),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 5)
}

# RandomizedSearchCV para XGBoost
xgb_random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=xgb_param_dist,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

xgb_random_search.fit(X_numerico_df, y)
print("Best parameters for XGBoost:", xgb_random_search.best_params_)

# Avaliação no conjunto de teste
xgb_best_model = xgb_random_search.best_estimator_
y_pred_xgb = xgb_best_model.predict(X_test_df)
y_pred_xgb

In [106]:
predictions = y_pred_xgb
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}



# Garantir que as previsões são mapeadas para o dicionário
predictions = [reverse_mapping.get(int(round(pred)), 'Unknown') for pred in predictions]

In [104]:
data = pd.DataFrame({
    'RowId': np.arange(1, len(predictions) + 1), 
    'Result': predictions
})

data.to_csv('XGBoostafterFS2.csv', index=False)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Instanciar o modelo KNN
knn = KNeighborsClassifier()

# Espaço de hiperparâmetros para o GridSearchCV
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Testar diferentes números de vizinhos
    'weights': ['uniform', 'distance'],  # Pesos
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Métrica de distância
}

# Configuração do GridSearchCV
knn_grid_search = GridSearchCV(
    estimator=knn,
    param_grid=knn_param_grid,
    scoring='f1_weighted',  # Ou outra métrica relevante (accuracy, recall, etc.)
    cv=5,  # Número de folds na validação cruzada
    verbose=2,
    n_jobs=-1  # Paralelizar para acelerar o treinamento
)

# Ajustar o modelo aos dados de treino
knn_grid_search.fit(X_numerico_df, y)

# Melhor conjunto de hiperparâmetros
print("Best parameters for KNN:", knn_grid_search.best_params_)


# Melhor modelo encontrado no GridSearch
knn_best_model = knn_grid_search.best_estimator_

# Previsões no conjunto de teste
y_pred_knn = knn_best_model.predict(X_test_df)
predictions = y_pred_knn

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters for KNN: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}


c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [10]:
predictions = y_pred_knn
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}



# Garantir que as previsões são mapeadas para o dicionário
predictions = [reverse_mapping.get(int(round(pred)), 'Unknown') for pred in predictions]

In [11]:
data = pd.DataFrame({
    'RowId': np.arange(1, len(predictions) + 1), 
    'Result': predictions
})

data.to_csv('KNNafterFS3.csv', index=False)